## ClothCo

- ClothCo makes three types of clothing: shirts, shorts, and pants. ClothCo needs to rent the appropriate machinery to make each type of clothing.
- The machinery can only be rented weekly; not by unit produced.
- The weekly machinery rental cost, unit production cost, unit sales price, unit labor usage and unit cloth usage for each type of clothing are given below.

<center>

| Type | Rental |  Profit | Labor (hr) | Cloth (sq. yd.) |
| ---- | ---- | ---- | ---- | ---- |
| Shirt | 200 | 6 | 3 | 4 |
| Shorts | 150 | 4 | 2 | 3 |
| Pants | 100 | 7 | 6 | 4 |

</center>

- Each week, 150 labor hours and 160 square yards of cloth are available.
- How should Clothco plan their weekly production to maximize their profit?

In [1]:
#Set up the data
rentalcosts = [200; 150; 100]
profit = [6; 4; 7]

labor = [3; 2; 6]
cloth = [4; 3; 4]

laboravail = 150
clothavail = 160

Mvec = [40; 54; 25]
;

In [8]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

#x tracks how much of clothing type i is made
@variable(m, x[1:3] >= 0)
#z tracks if any of clothing type i is made
@variable(m, z[1:3], Bin)

#Labor and cloth constraints
@constraint(m, laborcon, sum(labor[i]*x[i] for i in 1:3) <= laboravail)
@constraint(m, clothcon, sum(cloth[i]*x[i] for i in 1:3) <= clothavail)

#Big M constraints
@constraint(m, bigMcon[i in 1:3], x[i] <= Mvec[i]*z[i])

#Objective, profit minus fixed costs
@objective(m, Max, sum(profit[i]*x[i] for i in 1:3) - sum(rentalcosts[i]*z[i] for i in 1:3))

print(m)

Max 6 x[1] + 4 x[2] + 7 x[3] - 200 z[1] - 150 z[2] - 100 z[3]
Subject to
 laborcon : 3 x[1] + 2 x[2] + 6 x[3] ≤ 150
 clothcon : 4 x[1] + 3 x[2] + 4 x[3] ≤ 160
 bigMcon[1] : x[1] - 40 z[1] ≤ 0
 bigMcon[2] : x[2] - 54 z[2] ≤ 0
 bigMcon[3] : x[3] - 25 z[3] ≤ 0
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 z[1] binary
 z[2] binary
 z[3] binary


In [9]:
optimize!(m)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
5 rows, 6 cols, 12 nonzeros
5 rows, 6 cols, 12 nonzeros

Solving MIP model with:
   5 rows
   6 cols (3 binary, 0 integer, 0 implied int., 3 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   628.3333333     -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   628.3333333     -15             4288.89%        0      0      0         0     0.0s
         0       0         0   0.00%   81.75           -15              645.00%        0      0      0         2     0.0s

Solving report
  Status            Optimal
  Primal bound      75
  Dual bound        75
  Gap               0% (tolerance: 0.01%)
 

In [10]:
@show value.(x)
@show value.(z)
@show objective_value(m)

value.(x) = [0.0, 0.0, 25.0]
value.(z) = [0.0, 0.0, 1.0]
objective_value(m) = 75.0


75.0